In [1]:
import cv2
import torch
import pygame
import random
import numpy as np
import sys
sys.path.append('/home/jhkim/deepLearning/yolo/ultralytics')  # 클론한 ultralytics 디렉토리 경로
import ultralytics
from ultralytics import YOLO
"""
from IPython.display import display, Image
import IPython.display as ipd
"""
ultralytics.checks()

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12037MiB)
Setup complete ✅ (12 CPUs, 30.8 GB RAM, 1269.5/1832.2 GB disk)


In [2]:
# 모델 로드
model = YOLO("yolo11n-pose.pt")  # 포즈 모델

# 웹캠 열기
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO 모델로 포즈 추정
    results = model(frame)

    # Access the results
    """
    for result in results:
        xy = result.keypoints.xy  # x and y coordinates
        xyn = result.keypoints.xyn  # normalized
        kpts = result.keypoints.data  # x, y, visibility (if available)
    """
    result = results[0]
    keypoints = result.keypoints
    nb_detected = keypoints.shape[0]
    nb_detected = len(result.boxes)  # 감지된 바운딩 박스 개수

    # 이미지에 박스 삽입
    res = result.plot() # 각 관절 박스

    #show counter of detected people
    text = f'Human detected: {nb_detected}'
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    color = (0, 255, 0) 
    thickness = 2
    position = (50, 50)
    cv2.putText(res, text, position, font, font_scale, color, thickness, cv2.LINE_AA)
    
    """
    0: 코 1: 왼쪽 눈 2: 오른쪽 눈 3: 왼쪽 귀 4: 오른쪽 귀 5: 왼쪽 어깨 6: 오른쪽 어깨 
    7: 왼쪽 팔꿈치 8: 오른쪽 팔꿈치 9: 왼쪽 손목 10: 오른쪽 손목 11: 왼쪽 엉덩이 12: 오른쪽 엉덩이 
    13: 왼쪽 무릎 14: 오른쪽 무릎 15: 왼쪽 발목 16: 오른쪽 발목
    """
    confidence_threshold = 0.5
    left_shoulder_y = 0; 
    left_wrist_y = 0;
    right_shoulder_y = 0;
    right_wrist_y = 0;
    left_up = []
    right_up = []
    detected_count = 0;
    
    kpts = result.keypoints.data
    # print(f'kpts...[1] {kpts}')
    
    if nb_detected > 0:
        for i in range(nb_detected):
            # kpts = result.keypoints.data
            # print(f'kpts...[2] {kpts}')
            # print(kpts.shape)  # 텐서의 크기 출력
            
            left_shoulder_x, left_shoulder_y, left_shoulder_visibility = kpts[i, 5]
            left_wrist_x, left_wrist_y, left_wrist_visibility = kpts[i, 9]
            right_shoulder_x, right_shoulder_y, right_shoulder_visibility = kpts[i, 6]
            right_wrist_x, right_wrist_y, right_wrist_visibility = kpts[i, 10]
            
            # 왼쪽 어깨와 왼쪽 손목의 y 좌표 비교
            if left_shoulder_visibility < confidence_threshold or left_wrist_visibility < confidence_threshold:
                left_shoulder_y = 0
                left_wrist_y = 0
                
            # 오른쪽 어깨와 오른쪽 손목의 y 좌표 비교
            if right_shoulder_visibility < confidence_threshold or right_wrist_visibility < confidence_threshold:
                right_shoulder_y = 0
                right_wrist_y = 0
            
            # 왼쪽 손목이 왼쪽 어깨보다 위에 있는지 체크
            if left_wrist_y < left_shoulder_y:
                left_up.append(1)
            else:
                left_up.append(0)
            # 오른쪽 손목이 오른쪽 어깨보다 위에 있는지 체크
            if right_wrist_y < right_shoulder_y:
                right_up.append(1)
            else:
                right_up.append(0)
            text = f'{detected_count+1} : Left({left_up[detected_count]}) Right({right_up[detected_count]})'
            detected_count += 1
            position = (position[0], position[1] + 30) 
            cv2.putText(res, text, position, font, font_scale, color, thickness, cv2.LINE_AA)
            
    # 결과 출력
    # cv2.imshow('Pose Detection', frame)
    cv2.imshow('Pose Detection', res)
    
    # 'q' 키를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 [jhkim] 1 person, 32.8ms
Speed: 2.2ms preprocess, 32.8ms inference, 118.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 4.4ms
Speed: 0.8ms preprocess, 4.4ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 3.9ms
Speed: 0.6ms preprocess, 3.9ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 3.6ms
Speed: 0.5ms preprocess, 3.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)



qt.qpa.xcb: X server does not support XInput 2
failed to get the current screen resources
QObject::moveToThread: Current thread (0x22268740) is not the object's thread (0x32c875c0).
Cannot move to target thread (0x22268740)

QObject::moveToThread: Current thread (0x22268740) is not the object's thread (0x32c875c0).
Cannot move to target thread (0x22268740)

QObject::moveToThread: Current thread (0x22268740) is not the object's thread (0x32c875c0).
Cannot move to target thread (0x22268740)

QObject::moveToThread: Current thread (0x22268740) is not the object's thread (0x32c875c0).
Cannot move to target thread (0x22268740)

QObject::moveToThread: Current thread (0x22268740) is not the object's thread (0x32c875c0).
Cannot move to target thread (0x22268740)

QObject::moveToThread: Current thread (0x22268740) is not the object's thread (0x32c875c0).
Cannot move to target thread (0x22268740)

QObject::moveToThread: Current thread (0x22268740) is not the object's thread (0x32c875c0).
Cannot m

0: 480x640 [jhkim] 1 person, 3.9ms
Speed: 0.6ms preprocess, 3.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 3.7ms
Speed: 0.5ms preprocess, 3.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 5.3ms
Speed: 0.8ms preprocess, 5.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 3.6ms
Speed: 0.6ms preprocess, 3.6ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 3.5ms
Speed: 0.6ms preprocess, 3.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 3.5ms
Speed: 0.6ms preprocess, 3.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 3.5ms
Speed: 0.5ms preprocess, 3.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 [jhkim] 1 person, 3.5ms
Speed: 0.5ms preprocess, 3.5ms inference,